In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [4]:
import import_ipynb

ModuleNotFoundError: No module named 'import_ipynb'

In [3]:
from tf.keras.layers import Dense

ModuleNotFoundError: No module named 'tf'

In [3]:
import matplotlib
import pathlib
import glob

!pip install mne
import mne
print(mne.__version__)

from scipy.io import loadmat, savemat
import os
import numpy as np

  Using cached mne-0.24.1-py3-none-any.whl (7.4 MB)
0.24.1


In [4]:
def get_file_trials(file_path):
    """
    Utility function to get trial data from a .mat file
    
    Args:
    file_path: String; Location of the matlab file
    ----------
    Returns:
    trials: nparray; trials from the .mat file
    """
    mat = loadmat(file_path)
    trials = mat["trial"][0]
    return np.array([a for a in trials])

In [5]:
def get_data(subject_files_path):
    """
    A function to read data from all subjects and concatenate it
    
    Args:
    ----------
    subject_files_path: String; Location of where subject files are located
    
    Returns:
    ----------
    final_data: nparray; contains concatenated data from all trials
    final_y: nparray; contains corresponding labels
    
    """
    labels = ["breath", "feet", "future", "worry"]
    l_trial_data = {}
    for l in labels:
        l_trial_data[l] = []
    for files in glob.glob(subject_files_path+str("/subject*")):
        for file in glob.glob(files + str("/*")):
            for label in labels:
                if(file.split("\\")[-1] == f"{label}_cleaned_ica_data.mat"):
                    l_trial_data[label].append(get_file_trials(file))
    trial_data = {}
    for l in labels:
        trial_data[l] = np.concatenate(l_trial_data[l], axis = 0)
    y = {}
    for i in range(len(labels)):
        y[labels[i]] = np.full(trial_data[labels[i]].shape[0],i)
    final_data = np.concatenate([i for i in trial_data.values()], axis = 0)
    final_y = np.concatenate([i for i in y.values()], axis = 0)
    return final_data, final_y

In [6]:
X, y = get_data("meditation_eeg/data")

In [7]:
# np.savez('meditation_data', X = X, y = y)

In [8]:
# loadfile = np.load("meditation_data.npz")
# X = loadfile["X"]
# y = loadfile["y"]
# del loadfile

In [9]:
print(X.shape)
print(y.shape)

(24756, 32, 513)
(24756,)


In [10]:
# data = np.zeros((X.shape[0], X.shape[2], X.shape[1]))
# for i in range(X.shape[0]):
#     data[i] = np.transpose(X[i])

In [11]:
# data.shape

In [12]:
list(set(y))

[0, 1, 2, 3]

In [13]:
y

array([0, 0, 0, ..., 3, 3, 3])

In [21]:
num_classes = 4
y = keras.utils.to_categorical(y, num_classes)
ymlp = y.copy()
ylstm = y.copy()
ycnn = y.copy()
# yab10_1[0]
print(y[:10])

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [22]:
train_test_split = int(0.8 * len(X))
print(train_test_split)

x_train = X[:train_test_split]
y_train = y[:train_test_split]

x_test = X[train_test_split:]
y_test = y[train_test_split:]

19804


In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
inp_shape = (32, 513)
ip = keras.Input(shape = inp_shape)
conv1 = keras.layers.Conv1D(32, 3,)(ip)
rel = keras.layers.LeakyReLU()(conv1)
conv2 = keras.layers.Conv1D(16, 3,)(rel)
rel2 = keras.layers.LeakyReLU()(conv2)
# lstm = tf.keras.layers.LSTM(16, return_sequences=True)(ip)
flatten = keras.layers.Flatten()(rel2)
dense1 = keras.layers.Dense(units = 32, kernel_initializer = 'random_normal')(flatten)
leakyRelu = keras.layers.LeakyReLU()(dense1)
drop1 = keras.layers.Dropout(.4)(leakyRelu)
dense2 = keras.layers.Dense(units = 16, kernel_initializer = 'random_normal')(drop1)
leakyRelu2 = keras.layers.LeakyReLU()(dense2)
drop2 = keras.layers.Dropout(.4)(dense2)
out = keras.layers.Dense(4, activation = 'softmax')(drop2)

cnn2 = keras.Model(inputs = ip, outputs = out)
cnn2.summary()

In [30]:
# # Compiling the model
# cnn2_earlystop = EarlyStopping(patience = 10, min_delta = 0.001, monitor = 'val_acc')

# path = 'cnn2_checkpoint/checkpoint_{epoch:02d}';
# cnn2_checkpoint = ModelCheckpoint(filepath = path,
#                             verbose = 1,
#                             monitor = 'val_acc',
#                             save_freq = 'epoch',
#                             save_best_only = True,
#                             save_weights_only = True)

cnn2.compile(loss = 'categorical_crossentropy', metrics = ['acc'], optimizer = 'adam')

In [32]:
cnn2_history = cnn2.fit(x_train, y_train,
                        validation_data = (x_test, y_test),
#                         steps_per_epoch= 449,
                        batch_size = 32,
                        epochs = 30,
#                         callbacks = [cnn2_checkpoint],
                        )
                         #Fitting the model

Epoch 1/30
619/619 [==============================] - 22s 32ms/step - loss: 1.4962 - acc: 0.3049 - val_loss: 2.8387 - val_acc: 0.0000e+00
Epoch 2/30
619/619 [==============================] - 7s 11ms/step - loss: 1.2872 - acc: 0.3168 - val_loss: 2.6647 - val_acc: 2.0194e-04
Epoch 3/30
619/619 [==============================] - 7s 11ms/step - loss: 1.2735 - acc: 0.3124 - val_loss: 2.6925 - val_acc: 0.0000e+00
Epoch 4/30
619/619 [==============================] - 6s 10ms/step - loss: 1.2643 - acc: 0.3169 - val_loss: 2.8537 - val_acc: 0.0000e+00
Epoch 5/30
619/619 [==============================] - 6s 10ms/step - loss: 1.2586 - acc: 0.3276 - val_loss: 2.9648 - val_acc: 0.0000e+00
Epoch 6/30
619/619 [==============================] - 7s 11ms/step - loss: 1.2475 - acc: 0.3408 - val_loss: 2.9772 - val_acc: 0.0026
Epoch 7/30
619/619 [==============================] - 7s 12ms/step - loss: 1.2352 - acc: 0.3536 - val_loss: 3.2260 - val_acc: 6.0582e-04
Epoch 8/30
619/619 [========================

KeyboardInterrupt: 

In [ ]:
#Plotting model history
plt.plot(cnn2_history.history['acc'])
plt.plot(cnn2_history.history['loss'])
plt.legend(['Training Accuracy', 'Training Loss'])
plt.title('Training History')
plt.ylabel('Value')
plt.xlabel('epochs')
plt.show()
print("Initial training accuracy with preprocessing is: ", cnn2_history.history['acc'][0])
print("Final training accuracy with preprocessing is: ", cnn2_history.history['acc'][-1])

print("Initial training loss with preprocessing is: ", cnn2_history.history['loss'][0])
print("Final training loss with preprocessing is: ", cnn2_history.history['loss'][-1])